# Model Registering

In [404]:
# Import required libraries
import numpy as np
import pandas as pd
import os
import shutil 
import requests

import mlflow
import json
import yaml

import warnings
warnings.filterwarnings("ignore")

## Preparation

In [405]:
def load_params():
    with open("../params.yaml") as f:
        params = yaml.safe_load(f)
    return params

# Load params
params = load_params()

# Load recipes data
data = pd.read_pickle(params["model_pipeline"]["recipe_path"])

In [411]:
# Load validation model metadata
with open("../validation_metadata.json", "r") as f:
    metadata = json.load(f)

metadata["run_id"]

'32cc39d3e1e24799886476ad91037e36'

In [395]:
runs = mlflow.search_runs(experiment_names=["FastText-Model"], order_by=["start_time ASC", "metrics.ndcg DESC", "metrics.rmse DESC", "metrics.f1_10 DESC"])
best_run = runs.iloc[0]
best_run["run_id"]


'29f4ae3f26834dfd939536d0b57838a9'

In [396]:
runs

,run_id,experiment_id,status,artifact_uri,start_time,end_time,metrics.MAP,metrics.Precision_10,metrics.MAE,metrics.RMSE,...,params.window,params.local_uri,params.vector_size,params.sg,params.w_maxsim,params.w_title,tags.mlflow.user,tags.mlflow.source.type,tags.mlflow.runName,tags.mlflow.source.name
0,29f4ae3f26834dfd939536d0b57838a9,300877920446874715,FINISHED,mlflow-artifacts:/300877920446874715/29f4ae3f2...,2025-08-31 01:07:49.682000+00:00,2025-08-31 01:08:27.433000+00:00,0.190818,0.339457,0.194025,0.226918,...,5,../mlflow/mlartifacts/300877920446874715/model...,100,0,0.7,0.1,alfredmastan,LOCAL,glamorous-hog-637,/opt/anaconda3/envs/food-rec/lib/python3.12/si...
1,32cc39d3e1e24799886476ad91037e36,300877920446874715,FINISHED,mlflow-artifacts:/300877920446874715/32cc39d3e...,2025-08-31 01:10:14.219000+00:00,2025-08-31 01:11:34.823000+00:00,0.190818,0.339457,0.194025,0.226918,...,5,../mlflow/mlartifacts/300877920446874715/model...,100,0,0.7,0.1,alfredmastan,LOCAL,serious-stoat-300,/opt/anaconda3/envs/food-rec/lib/python3.12/si...


In [398]:
with open("../registered_metadata.json", "r") as f:
    registered_metadata = json.load(f)
registered_metadata["run_id"]

'29f4ae3f26834dfd939536d0b57838a9'

## Retrieve and Register Best Model

In [ ]:
mlflow.set_tracking_uri("http://127.0.0.1:8080")

# Get the highest performance and oldest run
runs = mlflow.search_runs(experiment_names=["FastText-Model"], order_by=["start_time ASC", "metrics.ndcg DESC", "metrics.rmse DESC", "metrics.f1_10 DESC"])
best_run = runs.iloc[0]

def main():
    # Check if any registered model exists
    if mlflow.search_registered_models():
        # Get current registered model metadata
        with open("../registered_metadata.json", "r") as f:
            registered_metadata = json.load(f)
        
        if best_run["run_id"] == registered_metadata["run_id"]:
            print(f"{'='*20} Current registered model is the best! {'='*20}")
            return

    # Load validation model metadata
    with open("../validation_metadata.json", "r") as f:
        metadata = json.load(f)
    
    # Save registered model metadata
    with open("../registered_metadata.json", "w") as f:
        f.write(json.dumps(metadata, indent=4))
        print("SAVED")
    
    # Register best model to MLFlow
    with mlflow.start_run(run_id=best_run.run_id):
        mlflow.register_model(
            model_uri=metadata["server_uri"],
            name="fasttext_model"
        )

    print(f"{'='*20} New best model registered! {'='*20}")

if __name__ == "__main__":
    main()

==================== Current registered model is the best! ====================


### API Call Test (Run Server Beforehand)

In [400]:
# params = {"query": ["rice", "beef", "meat"]}
# response = requests.post(f"http://localhost:8000/recommend/", params=params)
# np.asarray(json.loads(response.json()))

### Delete Old Runs and Artifacts (Based on Performance)

In [401]:
runs

,run_id,experiment_id,status,artifact_uri,start_time,end_time,metrics.MAP,metrics.Precision_10,metrics.MAE,metrics.RMSE,...,params.window,params.local_uri,params.vector_size,params.sg,params.w_maxsim,params.w_title,tags.mlflow.user,tags.mlflow.source.type,tags.mlflow.runName,tags.mlflow.source.name
0,29f4ae3f26834dfd939536d0b57838a9,300877920446874715,FINISHED,mlflow-artifacts:/300877920446874715/29f4ae3f2...,2025-08-31 01:07:49.682000+00:00,2025-08-31 01:08:27.433000+00:00,0.190818,0.339457,0.194025,0.226918,...,5,../mlflow/mlartifacts/300877920446874715/model...,100,0,0.7,0.1,alfredmastan,LOCAL,glamorous-hog-637,/opt/anaconda3/envs/food-rec/lib/python3.12/si...
1,32cc39d3e1e24799886476ad91037e36,300877920446874715,FINISHED,mlflow-artifacts:/300877920446874715/32cc39d3e...,2025-08-31 01:10:14.219000+00:00,2025-08-31 01:11:34.823000+00:00,0.190818,0.339457,0.194025,0.226918,...,5,../mlflow/mlartifacts/300877920446874715/model...,100,0,0.7,0.1,alfredmastan,LOCAL,serious-stoat-300,/opt/anaconda3/envs/food-rec/lib/python3.12/si...


In [ ]:
deprecate_runs = runs.iloc[params["model_register"]["max_runs"]:]

for index, run in deprecate_runs.iterrows():
    # Delete run from MLFlow
    id = run["run_id"]
    mlflow.delete_run(id)

    # Delete run from local file system
    run_path = os.path.join("../mlflow/mlruns", run["experiment_id"], run["run_id"])
    if os.path.exists(run_path):
        print(f"Deleting local run directory: {run_path}")
        shutil.rmtree(run_path)

    # Delete artifact from local file system
    artifact_path = os.path.join(*run["params.local_uri"].split("/")[:-1]) 
    
    if os.path.exists(artifact_path):
        print(f"Deleting local model artifacts: {artifact_path}")
        shutil.rmtree(artifact_path)